## Language Analysis of Alexithymic Discourse

<hr>

Alexithymic Language Project / raul@psicobotica.com / V2 release (sept 2020)

<hr>

### Personality traits detection

There are many mechanisms by which personality traits of the author might be inferred from their text. Here, instead of training our own model (which requires extensive annotated data), we use IBM Personality Insights API. 

- Establish connection with IBM Personality Insights API. 
- sdfsd 

<hr>

- [IBM Personality Insights API](https://www.ibm.com/watson/services/personality-insights/)
- [Big Five Personality Facets](https://github.com/watson-developer-cloud/doc-tutorial-downloads/blob/master/personality-insights/Personality-Insights-Facet-Characteristics.pdf)
- [Big Five Personality Dimensions](https://github.com/watson-developer-cloud/doc-tutorial-downloads/blob/master/personality-insights/Personality-Insights-Dimension-Characteristics.pdf)
- [Consumer Consumption Preference Prediction](https://github.com/watson-developer-cloud/doc-tutorial-downloads/blob/master/personality-insights/Improving-Consumption-Preferences-Accuracy.pdf)


### Using the Watson Developer Cloud Python SDK
- [Python SDK](https://github.com/watson-developer-cloud/python-sdk)
- [API Docs for Python](https://cloud.ibm.com/apidocs/personality-insights?code=python)

Using my own IBM Cloud account. 

Installing IBM Cloud python SDK


In [3]:
# ! pip install --upgrade ibm-watson

  Created wheel for ibm-watson: filename=ibm_watson-4.7.1-py3-none-any.whl size=379136 sha256=21610ae36a3b6eb8d19c5bb0416b2a2ae28a1101e2e5b820594abf1cbd452660
  Stored in directory: c:\users\array\appdata\local\pip\cache\wheels\63\0c\16\b2af79aace901934f32f1503a95b338c7bdca6ab2cfa0881e6
  Created wheel for ibm-cloud-sdk-core: filename=ibm_cloud_sdk_core-1.7.3-py3-none-any.whl size=45948 sha256=9b7d8e13169a143ba8c9d87b931e6ecd30e00a10209cd5cd7f0dd2673904fdb9
  Stored in directory: c:\users\array\appdata\local\pip\cache\wheels\c6\5f\fb\c2daef3706ded7e718641a60d097be3116a6b70a8f236753f2
Successfully built ibm-watson ibm-cloud-sdk-core
  Attempting uninstall: ibm-cloud-sdk-core
    Found existing installation: ibm-cloud-sdk-core 0.4.2
    Uninstalling ibm-cloud-sdk-core-0.4.2:
      Successfully uninstalled ibm-cloud-sdk-core-0.4.2
  Attempting uninstall: ibm-watson
    Found existing installation: ibm-watson 3.0.4
    Uninstalling ibm-watson-3.0.4:
      Successfully uninstalled ibm-watso

### IBM Cloud Authentication and endpoint

In [46]:
# Following the IBM cloud official example at:
# https://github.com/watson-developer-cloud/python-sdk/blob/master/examples/personality_insights_v3.py 
import pandas as pd
import json
import os
from os.path import join
from ibm_watson import PersonalityInsightsV3
import csv
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator

In [34]:
# Create the API Key
# API key stored in local file apikey.json
with open('apikey.json') as f:
    apikeydata = json.load(f)

In [38]:
# apikeydata.get('PI_key')

In [17]:
# Create a Personality Insight Price Plan (Lite: 1000 calls per month free)
# Select Location. 
# Personality Insights-ro activated
# Service credentials: get it from the list of resources. 

In [39]:
# Using London location (the only one in Europe): 
# https://api.eu-gb.personality-insights.watson.cloud.ibm.com 
# My instance: https://api.eu-gb.personality-insights.watson.cloud.ibm.com/instances/8d60aa18-abce-43f2-8ee0-bfa00b8a1140 

# authenticator = IAMAuthenticator(apikeydata.get('apikey'))
authenticator = IAMAuthenticator(apikeydata.get('PI_key'))
personality_insights = PersonalityInsightsV3(
    version='2017-10-13',  # as per last version documented in API Docs
    authenticator=authenticator
)

personality_insights.set_service_url('https://api.eu-gb.personality-insights.watson.cloud.ibm.com/instances/8d60aa18-abce-43f2-8ee0-bfa00b8a1140')

In [40]:
# Let's first try the API with a simple text 
test_Text = "Aunque el estudio de la mente y su relación con el cerebro es un fenómeno con una larga trayectoria histórica, el enfoque moderno interdisciplinar denominado neurociencia cognitiva aparece en 1976 de la mano de Michael Gazzaniga y George Miller. Durante las cuatro últimas décadas la ciencia del cerebro ha experimentado muchos avances significativos y el área multidisciplinar de la neurociencia cognitiva también ha ido experimentando cambios. Inicialmente, la neurociencia cognitiva se entendía como la colaboración entre el estudio de la mente (psicología) y el estudio del sistema nervioso (neurociencias). En las primeras etapas, el objetivo primordial era entender cómo los procesos neurobiológicos daban lugar a los procesos cognitivos. Es decir, comprender la cognición a través de sus correlatos neurobiológicos. Sin embargo, la mente no se puede explicar completamente desde la dimensión cognitiva (procesamiento de la información), sino que, como bien sabemos hoy en día, la emoción juega también un papel primordial. Por este motivo, algunos autores empezaron a habla de neurociencia afectiva, para resaltar el hecho de que el estudio del sistema nervioso también ha de ir dirigido a comprender la dimensión emocional de la mente."
len(test_Text.split())

186

In [41]:
# Getting the results as CSV:
from ibm_watson import ApiException
try:
    # Invoke a Personality Insights method
    response = personality_insights.profile(
        test_Text,
        content_type='text/plain',
        content_language='es',
        accept_language='en',
        consumption_preferences=True,
        raw_scores=True,
        accept='text/csv',
        csv_headers=True).get_result()
except ApiException as ex:
    print("Method failed with status code " + str(ex.code) + ": " + ex.message)

In [114]:
df = pd.DataFrame()

profile = response.content
cr = csv.reader(profile.decode('utf-8').splitlines())
my_list = list(cr)
for row in my_list:
    print(row[0:10])

AttributeError: 'dict' object has no attribute 'content'

In [63]:
import io
df = pd.read_csv(io.StringIO(profile.decode('utf-8')), lineterminator='\n')

In [71]:
df

,big5_agreeableness,facet_altruism,facet_cooperation,facet_modesty,facet_morality,facet_sympathy,facet_trust,big5_conscientiousness,facet_achievement_striving,facet_cautiousness,...,consumption_preferences_movie_action,consumption_preferences_movie_documentary,consumption_preferences_read_frequency,consumption_preferences_books_entertainment_magazines,consumption_preferences_books_non_fiction,consumption_preferences_books_financial_investing,consumption_preferences_books_autobiographies,consumption_preferences_volunteer,consumption_preferences_concerned_environment,consumption_preferences_start_business
0,0.876784,0.681975,0.770881,0.274411,0.754819,0.485524,0.693573,0.863901,0.423906,0.666144,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.5,1.0


In [72]:
# Getting the results as JSON:
from ibm_watson import ApiException
try:
    # Invoke a Personality Insights method
    response = personality_insights.profile(
        test_Text,
        content_type='text/plain',
        content_language='es',
        accept_language='en',
        consumption_preferences=True,
        raw_scores=True,
        accept='application/json').get_result()
except ApiException as ex:
    print("Method failed with status code " + str(ex.code) + ": " + ex.message)

In [73]:
print(json.dumps(response, indent=2))

{
  "word_count": 205,
  "word_count_message": "There were 205 words in the input. We need a minimum of 600, preferably 1,200 or more, to compute statistically significant estimates",
  "processed_language": "es",
  "personality": [
    {
      "trait_id": "big5_openness",
      "name": "Openness",
      "category": "personality",
      "percentile": 0.948309173503183,
      "raw_score": 0.6696244499549608,
      "significant": true,
      "children": [
        {
          "trait_id": "facet_adventurousness",
          "name": "Adventurousness",
          "category": "personality",
          "percentile": 0.80270805543852,
          "raw_score": 0.504724921504605,
          "significant": true
        },
        {
          "trait_id": "facet_artistic_interests",
          "name": "Artistic interests",
          "category": "personality",
          "percentile": 0.963600473898098,
          "raw_score": 0.6892566878755113,
          "significant": true
        },
        {
          "t

In [86]:
for i in response['personality']:
    print(i['name'])

Openness
Conscientiousness
Extraversion
Agreeableness
Emotional range


In [113]:
# Personality traits percentiles:
for i in range(len(response['personality'])):
    print(response['personality'][i]['trait_id'] + 
          " --> %f" % (response['personality'][i]['percentile']))
    for j in range(len(response['personality'][i]['children'])):
        print(" > " + response['personality'][i]['children'][j]['trait_id'] +
             " --> %f" % response['personality'][i]['children'][j]['percentile'])

big5_openness --> 0.948309
 > facet_adventurousness --> 0.802708
 > facet_artistic_interests --> 0.963600
 > facet_emotionality --> 0.759525
 > facet_imagination --> 0.788288
 > facet_intellect --> 0.868290
 > facet_liberalism --> 0.840628
big5_conscientiousness --> 0.863901
 > facet_achievement_striving --> 0.423906
 > facet_cautiousness --> 0.666144
 > facet_dutifulness --> 0.832666
 > facet_orderliness --> 0.894099
 > facet_self_discipline --> 0.866430
 > facet_self_efficacy --> 0.690610
big5_extraversion --> 0.644563
 > facet_activity_level --> 0.643835
 > facet_assertiveness --> 0.636832
 > facet_cheerfulness --> 0.228924
 > facet_excitement_seeking --> 0.592070
 > facet_friendliness --> 0.555696
 > facet_gregariousness --> 0.197778
big5_agreeableness --> 0.876784
 > facet_altruism --> 0.681975
 > facet_cooperation --> 0.770881
 > facet_modesty --> 0.274411
 > facet_morality --> 0.754819
 > facet_sympathy --> 0.485524
 > facet_trust --> 0.693573
big5_neuroticism --> 0.422815
 > fa

In [121]:
# Personality traits raw scores:
for i in range(len(response['personality'])):
    print(response['personality'][i]['trait_id'] + 
          " --> %f" % (response['personality'][i]['raw_score']))
    for j in range(len(response['personality'][i]['children'])):
        print(" > " + response['personality'][i]['children'][j]['trait_id'] +
             " --> %f" % response['personality'][i]['children'][j]['raw_score'])

big5_openness --> 0.669624
 > facet_adventurousness --> 0.504725
 > facet_artistic_interests --> 0.689257
 > facet_emotionality --> 0.633692
 > facet_imagination --> 0.832148
 > facet_intellect --> 0.573205
 > facet_liberalism --> 0.522924
big5_conscientiousness --> 0.567220
 > facet_achievement_striving --> 0.516337
 > facet_cautiousness --> 0.369563
 > facet_dutifulness --> 0.637402
 > facet_orderliness --> 0.407455
 > facet_self_discipline --> 0.539749
 > facet_self_efficacy --> 0.720451
big5_extraversion --> 0.567928
 > facet_activity_level --> 0.519684
 > facet_assertiveness --> 0.627894
 > facet_cheerfulness --> 0.633518
 > facet_excitement_seeking --> 0.635550
 > facet_friendliness --> 0.588775
 > facet_gregariousness --> 0.484269
big5_agreeableness --> 0.630361
 > facet_altruism --> 0.681486
 > facet_cooperation --> 0.515986
 > facet_modesty --> 0.366669
 > facet_morality --> 0.577618
 > facet_sympathy --> 0.632327
 > facet_trust --> 0.579026
big5_neuroticism --> 0.642880
 > fa

In [119]:
# Needs percentiles:
for i in range(len(response['needs'])):
    print(response['needs'][i]['trait_id'] + 
          " --> %f" % (response['needs'][i]['percentile']))

need_challenge --> 0.560802
need_closeness --> 0.769558
need_curiosity --> 0.656694
need_excitement --> 0.109382
need_harmony --> 0.952191
need_ideal --> 0.550224
need_liberty --> 0.451310
need_love --> 0.458311
need_practicality --> 0.862161
need_self_expression --> 0.387077
need_stability --> 0.902629
need_structure --> 0.667573


In [120]:
# Needs raw-scores:
for i in range(len(response['needs'])):
    print(response['needs'][i]['trait_id'] + 
          " --> %f" % (response['needs'][i]['raw_score']))

need_challenge --> 0.748814
need_closeness --> 0.788978
need_curiosity --> 0.790934
need_excitement --> 0.695507
need_harmony --> 0.865426
need_ideal --> 0.663528
need_liberty --> 0.742571
need_love --> 0.760919
need_practicality --> 0.785543
need_self_expression --> 0.642778
need_stability --> 0.716149
need_structure --> 0.668686


In [122]:
# Values percentiles:
for i in range(len(response['values'])):
    print(response['values'][i]['trait_id'] + 
          " --> %f" % (response['values'][i]['percentile']))

value_conservation --> 0.566254
value_openness_to_change --> 0.311321
value_hedonism --> 0.105587
value_self_enhancement --> 0.764340
value_self_transcendence --> 0.704713


In [123]:
# Values raw score:
for i in range(len(response['values'])):
    print(response['values'][i]['trait_id'] + 
          " --> %f" % (response['values'][i]['raw_score']))

value_conservation --> 0.669982
value_openness_to_change --> 0.783385
value_hedonism --> 0.795683
value_self_enhancement --> 0.700313
value_self_transcendence --> 0.849676


In [141]:
# Consumption Preferences
for i in range(len(response['consumption_preferences'])):
    print(response['consumption_preferences'][i]['consumption_preference_category_id'])
    for j in range(len(response['consumption_preferences'][i]['consumption_preferences'])):
        print(" > " + response['consumption_preferences'][i]['consumption_preferences'][j]['consumption_preference_id'] +
             " --> %.2f" % (response['consumption_preferences'][i]['consumption_preferences'][j]['score']))

consumption_preferences_shopping
 > consumption_preferences_automobile_ownership_cost --> 0.00
 > consumption_preferences_automobile_safety --> 1.00
 > consumption_preferences_clothes_quality --> 0.00
 > consumption_preferences_clothes_style --> 0.00
 > consumption_preferences_clothes_comfort --> 0.00
 > consumption_preferences_influence_brand_name --> 0.50
 > consumption_preferences_influence_utility --> 0.50
 > consumption_preferences_influence_online_ads --> 1.00
 > consumption_preferences_influence_social_media --> 1.00
 > consumption_preferences_influence_family_members --> 1.00
 > consumption_preferences_spur_of_moment --> 0.50
 > consumption_preferences_credit_card_payment --> 1.00
consumption_preferences_health_and_activity
 > consumption_preferences_eat_out --> 0.00
 > consumption_preferences_gym_membership --> 0.00
 > consumption_preferences_outdoor --> 0.50
consumption_preferences_environmental_concern
 > consumption_preferences_concerned_environment --> 0.50
consumption_pre